Imports

In [8]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
from textstat.textstat import textstatistics
from bs4 import BeautifulSoup
import requests
import csv

Reading the data from input excel file into a pandas dataframe

In [14]:
data = pd.read_excel("input.xlsx")
df = pd.DataFrame(data)
data = {"URL_ID" : [],"URL" : [],"articles" : []}
datalist = pd.DataFrame(data)

Scraping the article text from the links and creating text files for each URL_ID

In [15]:

for ind,row in df.iterrows() :
    
    try : 
        url = row['URL']
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        result = requests.get(url = url,headers = headers).text
        
        doc = BeautifulSoup(result, "html.parser")
        article = doc.find(class_ = "td-post-content")
        article_final = ""

        for ar in article.find_all("p") : 
            article_final += ar.text

        file = str(int(row["URL_ID"])) + ".txt"

        dd = {"URL_ID" : [int(row["URL_ID"])],"URL" : [row["URL"]],"articles" : [article_final]}
        dd1 = pd.DataFrame(dd)
        #datalist = datalist.append(dd,ignore_index=True)
        datalist = pd.concat([datalist,dd1],ignore_index=True)
        
        f = open(file,"w+")
        f.write(article_final)
        f.close()

    except :
        print("error in loading the page as its not available")

error in loading the page as its not available
error in loading the page as its not available
error in loading the page as its not available


Adding all the stopwords in a list using the given files 

In [11]:
stpwrds = []
f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001/StopWords/StopWords_Auditor.txt', 'r')
adtr = f.read().splitlines()
f.close()
adtr_l = [name.lower() for name in adtr]
stpwrds.extend(adtr_l)

f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001\StopWords\StopWords_Currencies.txt', 'r')
curr = f.read().splitlines()
f.close()
curr_s = [cur.split("|") for cur in curr]
curr_m = []
for i in curr_s :
    curr_m.append(i[0].strip().lower())
stpwrds.extend(curr_m)

f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001\StopWords\StopWords_DatesandNumbers.txt', 'r')
dnm = f.read().splitlines()
f.close()
dnm_s = [dn.split("|") for dn in dnm]
dnm_m = []
for i in dnm_s :
    dnm_m.append(i[0].strip().lower())
stpwrds.extend(dnm_m)

f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001\StopWords\StopWords_Generic.txt', 'r')
gen = f.read().splitlines()
f.close()
gen_l = [name.lower() for name in gen]
stpwrds.extend(gen_l)

f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001\StopWords\StopWords_GenericLong.txt', 'r')
gens = f.read().splitlines()
f.close()
gens_l = [name.lower() for name in gens]
stpwrds.extend(gens_l)

f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001\StopWords\StopWords_Geographic.txt', 'r')
gps = f.read().splitlines()
f.close()
gps_s = [dn.split("|") for dn in gps]
gps_m = []
for i in gps_s :
    gps_m.append(i[0].strip().lower())
stpwrds.extend(gps_m)

f = open('C:/Users/satya/Desktop/blackcoffrintern/StopWords-20221230T110845Z-001\StopWords\StopWords_Geographic.txt', 'r')
nms = f.read().splitlines()
f.close()
nms_s = [dn.split("|") for dn in nms]
nms_m = []
for i in nms_s :
    nms_m.append(i[0].strip().lower())
stpwrds.extend(nms_m)

In [22]:
df1 = datalist.copy()

,URL_ID,URL,articles
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,introduction“if anything kills over 10 million...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,"human minds, a fascination in itself carrying ..."
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,introductionai is rapidly evolving in the empl...
3,40.0,https://insights.blackcoffer.com/will-machine-...,“anything that could give rise to smarter-than...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,“machine intelligence is the last invention th...
...,...,...,...
106,146.0,https://insights.blackcoffer.com/blockchain-fo...,reconciling with the financial realities of an...
107,147.0,https://insights.blackcoffer.com/the-future-of...,an investment is a resource or thing procured ...
108,148.0,https://insights.blackcoffer.com/big-data-anal...,quality and affordable healthcare is a vision ...
109,149.0,https://insights.blackcoffer.com/business-anal...,analytics is a statistical scientific process ...


Making all the article text lowercase

In [21]:
df1['articles'] = df1['articles'].str.lower()

Removing punctuations and stopwords

In [24]:
def remove_punctuation(text):
      return re.sub(r'[^\w\s]',' ',text)

def remove_stopwords(text):
      text = ' '.join([word for word in text.split() if word not in (stpwrds)])
      return text

df1['articles'] = df1['articles'].apply(lambda x : remove_punctuation(x))
df1['articles'] = df1['articles'].apply(lambda x : remove_stopwords(x))

Loading the positive and negative dictionaries

In [25]:
f = open('C:/Users/satya/Desktop/blackcoffrintern/MasterDictionary-20221230T110842Z-001/MasterDictionary/positive-words.txt', 'r')
pos = f.read().splitlines()
f.close()

n = open('C:/Users/satya/Desktop/blackcoffrintern/MasterDictionary-20221230T110842Z-001/MasterDictionary/negative-words.txt', 'r')
neg = n.read().splitlines()
n.close()

function for computing indeces requiring cleaned text

In [26]:
def compute_indeces(text) :
    
    tokens = word_tokenize(text)
    
    pos_score = 0
    neg_score = 0
    pol_score = 0
    sub_score = 0
    wor_count = 0

    for token in tokens :
        if token in pos :
            pos_score += 1
        elif token in neg :
            neg_score += 1

    pol_score = (pos_score - neg_score)/((pos_score+neg_score)+0.000001)
    sub_score = (pos_score+neg_score)/((len(tokens)+0.000001))

    result = {"positive score" : pos_score,"negative score" : neg_score,"polarity score" : pol_score,"subjectivity score" : sub_score,"word count" : len(tokens)}

    return result

functions for computing other indeces

In [27]:
def word_count(text):
    sentences = break_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words

def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)

def syllables_count(word):
    return textstatistics().syllable_count(word)

def avg_sentence_length(text):
    words = word_count(text)
    sentences = sentence_count(text)
    average_sentence_length = float(words / sentences)
    return average_sentence_length

def sentence_count(text):
    sentences = break_sentences(text)
    return len(sentences)

def complex_words(text):
     
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)


def gunning_fog(text):
    per_cmplx_words = (complex_words(text) / word_count(text) * 100)
    grade = 0.4 * (avg_sentence_length(text) + per_cmplx_words)
    return grade

def syllable_count_word(text) :
    
    words = []
    sentences = break_sentences(text)
    syllable_count = 0
    for sentence in sentences:
        words += [str(token) for token in sentence]

    for word in words:
        syllable_count += syllables_count(word)
    
    return syllable_count

def avg_word_length(text) :
    ans = len(text)/word_count(text)
    return ans

def personal_pronouns(text) :
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(text)
    return len(pronouns)

In [28]:
pos_score = []
neg_score = []
pol_score = []
sub_score = []
avg_senlen = []
per_cmplx = []
fog_index = []
avg_wrdspsen = []
cmplx_wordcnt = []
wrd_cnt = []
syll_pwrd = []
per_pro = []
avg_wrdlen = []

In [29]:
for ind,row in df1.iterrows() :

    ans = compute_indeces(row["articles"])
    pos_score.append(ans["positive score"])
    neg_score.append(ans["negative score"])
    pol_score.append(ans["polarity score"])
    sub_score.append(ans["subjectivity score"])
    wrd_cnt.append(ans["word count"])

In [30]:
df2 = datalist.copy()
for ind,row in df2.iterrows() :
    asl = avg_sentence_length(row["articles"])
    pcw = (complex_words(row["articles"])/word_count(row["articles"]))*100
    fi = gunning_fog(row["articles"])
    anwps = avg_sentence_length(row["articles"])
    cpw = complex_words(row["articles"])
    scw = syllable_count_word(row["articles"])
    awl = avg_word_length(row["articles"])
    pp = personal_pronouns(row["articles"])
    
    avg_senlen.append(asl)
    per_cmplx.append(pcw)
    fog_index.append(fi)
    avg_wrdspsen.append(anwps)
    cmplx_wordcnt.append(cpw)
    syll_pwrd.append(scw)
    per_pro.append(pp)
    avg_wrdlen.append(awl)

Adding all the indeces in the final dataframe

In [31]:
df2.drop(['articles'],axis=1,inplace=True)
df2['POSITIVE SCORE'] = pos_score
df2['NEGATIVE SCORE'] = neg_score
df2['POLARITY SCORE'] = pol_score
df2['SUBJECTIVITY SCORE'] = sub_score
df2['AVG SENTENCE LENGTH'] = avg_senlen
df2['PERCENTAGE OF COMPLEX WORDS'] = per_cmplx
df2['FOG INDEX'] = fog_index
df2['AVERAGE NO OF WORDS PER SENTENCE'] = avg_wrdspsen
df2['COMPLEX WORD COUNT'] = cmplx_wordcnt
df2['WORD COUNT'] = wrd_cnt
df2['SYLLABLE PER WORD'] = syll_pwrd
df2['PERSONAL PRONOUNS'] = per_pro
df2['AVERAGE WORD LENGTH'] = avg_wrdlen

Saving the dataframe into a csv file

In [43]:
df2.to_csv("output.csv",index=False)